In [2]:
# Day 2 — Statistical evaluation scaffold + results for the Random-Walk (RW) baseline
# We will:
# (A) Compute block-bootstrap CIs for Accuracy, Weighted-F1, Macro-F1, MCC (pooled OOS and per split)
# (B) Prepare McNemar and Diebold–Mariano (DM) test utilities (to compare any model vs RW later)
# (C) Run them on RW where applicable (for RW vs RW, these yield neutral results; utilities will be reused tomorrow)

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [6]:
rw_preds_path = Path("Data/10_day_run/rw_predictions_all.csv")
splits_csv = Path("Data/10_day_run/rolling_splits.csv")
rw_preds = pd.read_csv(rw_preds_path)
splits_df = pd.read_csv(splits_csv)

In [8]:
# Ensure types
rw_preds['y_true'] = rw_preds['y_true'].astype(int)
rw_preds['y_pred'] = rw_preds['y_pred'].astype(int)
labels_order = [-1, 0, 1]

In [20]:
# -------------------- (A) Moving Block Bootstrap (MBB) --------------------
# For time series, we prefer block bootstrap to preserve local dependence.
# We'll implement a circular MBB:
# - Choose block length b (pooled: ~sqrt(n) ~= 11; use 10 as a clean choice)
# - Number of blocks k = ceil(n / b); sample starts uniformly and wrap around

def mbb_indices(n, block_len=10, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    k = int(np.ceil(n / block_len))
    starts = rng.integers(low=0, high=n, size=k)
    idx = []
    for s in starts:
        block = [(s + i) % n for i in range(block_len)]
        idx.extend(block)
    return np.array(idx[:n], dtype=int)

def metric_tuple(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    f1_w = f1_score(y_true, y_pred, average='weighted', labels=labels_order, zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', labels=labels_order, zero_division=0)
    return acc, f1_w, f1_macro, mcc

def mbb_ci(y_true, y_pred, block_len=10, B=2000, seed=42):
    rng = np.random.default_rng(seed)
    n = len(y_true)
    stats = []
    for _ in range(B):
        idx = mbb_indices(n, block_len=block_len, rng=rng)
        stats.append(metric_tuple(y_true[idx], y_pred[idx]))
    stats = np.array(stats)  # shape (B, 4)
    # point estimate
    pt = np.array(metric_tuple(y_true, y_pred))
    # percentile CIs
    lo = np.percentile(stats, 2.5, axis=0)
    hi = np.percentile(stats, 97.5, axis=0)
    return pt, lo, hi

In [21]:
# Pooled OOS metrics + CIs
y_true_all = rw_preds['y_true'].to_numpy()
y_pred_all = rw_preds['y_pred'].to_numpy()
pt_all, lo_all, hi_all = mbb_ci(y_true_all, y_pred_all, block_len=10, B=4000, seed=123)

pooled_ci_df = pd.DataFrame({
    "Metric": ["Accuracy", "Weighted F1", "Macro F1", "MCC"],
    "Point": pt_all,
    "CI Lower (95%)": lo_all,
    "CI Upper (95%)": hi_all,
    "n_weeks": [len(y_true_all)]*4
})

In [22]:
# Per-split CIs (smaller test size; use shorter blocks to fit, e.g., b=4)
per_split_rows = []
for _, r in splits_df.iterrows():
    sid = int(r['split_id'])
    sub = rw_preds[rw_preds['split_id'] == sid].copy()
    y_t = sub['y_true'].to_numpy()
    y_p = sub['y_pred'].to_numpy()
    b = 4 if len(sub) >= 8 else max(2, len(sub)//2)
    pt, lo, hi = mbb_ci(y_t, y_p, block_len=b, B=2000, seed=100+sid)
    for name, p, l, h in zip(["Accuracy", "Weighted F1", "Macro F1", "MCC"], pt, lo, hi):
        per_split_rows.append({
            "split_id": sid,
            "Metric": name,
            "Point": p,
            "CI Lower (95%)": l,
            "CI Upper (95%)": h,
            "n_weeks": len(sub)
        })
per_split_ci_df = pd.DataFrame(per_split_rows)

In [23]:
# -------------------- (B) McNemar test utility --------------------
# Given two models' predictions over the SAME weeks, test if their hit rates differ.
# Returns the chi-square stat and p-value (with continuity correction).

from math import fabs
from scipy.stats import chi2

def mcnemar_test(y_true, y_pred_a, y_pred_b):
    hit_a = (y_true == y_pred_a).astype(int)
    hit_b = (y_true == y_pred_b).astype(int)
    n01 = int(((hit_a == 0) & (hit_b == 1)).sum())
    n10 = int(((hit_a == 1) & (hit_b == 0)).sum())
    if (n01 + n10) == 0:
        return {"n01": n01, "n10": n10, "chi2": 0.0, "p_value": 1.0}
    from math import fabs
    from scipy.stats import chi2
    stat = (fabs(n01 - n10) - 1)**2 / (n01 + n10)
    p = 1 - chi2.cdf(stat, df=1)
    return {"n01": n01, "n10": n10, "chi2": float(stat), "p_value": float(p)}

In [24]:
# (Sanity) RW vs RW should be null
mcnemar_null = mcnemar_test(y_true_all, y_pred_all, y_pred_all)

In [25]:
# -------------------- (C) Diebold–Mariano test --------------------
# Compare loss series of two models. We'll use 0–1 loss by default.
# We implement a standard DM with Newey–West variance (lag = h-1; here horizon h=1 → lag=0).

import math

def dm_test(loss_a, loss_b, h=1, lag=None):
    d = np.array(loss_a) - np.array(loss_b)  # loss differential
    T = len(d)
    d_bar = d.mean()
    if lag is None:
        lag = h - 1  # for h=1, lag=0
    # Newey–West HAC variance estimate
    gamma0 = np.sum((d - d_bar)**2) / T
    var = gamma0
    for l in range(1, lag+1):
        cov = np.sum((d[l:] - d_bar) * (d[:-l] - d_bar)) / T
        w = 1 - l/(lag+1)
        var += 2 * w * cov
    var = var / T
    stat = d_bar / (np.sqrt(var) + 1e-12)
    # Two-sided p-value under asymptotic normality
    from scipy.stats import norm
    p = 2 * (1 - norm.cdf(abs(stat)))
    return {"dm_stat": stat, "p_value": p, "mean_diff": d_bar, "T": T}

In [26]:
# Example: DM null using RW vs RW (should be neutral)
loss_rw = (y_true_all != y_pred_all).astype(int)
dm_null = dm_test(loss_rw, loss_rw, h=1, lag=0)

In [27]:
print("McNemar (RW vs RW) sanity:", mcnemar_null)
print("DM (RW vs RW) sanity:", dm_null)

McNemar (RW vs RW) sanity: {'n01': 0, 'n10': 0, 'chi2': 0.0, 'p_value': 1.0}
DM (RW vs RW) sanity: {'dm_stat': np.float64(0.0), 'p_value': np.float64(1.0), 'mean_diff': np.float64(0.0), 'T': 120}


In [28]:
pooled_ci_df.head()

,Metric,Point,CI Lower (95%),CI Upper (95%),n_weeks
0,Accuracy,0.375000,0.300000,0.450000,120
1,Weighted F1,0.375262,0.301912,0.448265,120
2,Macro F1,0.374870,0.294655,0.440154,120
3,MCC,0.048128,-0.081927,0.150520,120


In [30]:
per_split_ci_df.head()

,split_id,Metric,Point,CI Lower (95%),CI Upper (95%),n_weeks
0,1,Accuracy,0.304348,0.130435,0.521739,23
1,1,Weighted F1,0.313043,0.130637,0.550741,23
2,1,Macro F1,0.207407,0.089717,0.314286,23
3,1,MCC,-0.270092,-0.592428,0.004119,23
4,2,Accuracy,0.304348,0.130435,0.521739,23


In [31]:
# Additional diagnostics for your write-up: class mix and confusion matrix for RW pooled OOS

from sklearn.metrics import confusion_matrix

y_true_all = rw_preds['y_true'].astype(int).to_numpy()
y_pred_all = rw_preds['y_pred'].astype(int).to_numpy()

labels_order = [-1, 0, 1]
cm = confusion_matrix(y_true_all, y_pred_all, labels=labels_order)
support = pd.Series(y_true_all).value_counts().reindex(labels_order).fillna(0).astype(int)

cm_df = pd.DataFrame(cm, index=[f"True {c}" for c in labels_order], columns=[f"Pred {c}" for c in labels_order])
support_df = pd.DataFrame({"Class": labels_order, "Support": support.values})

In [33]:
cm_df

,Pred -1,Pred 0,Pred 1
True -1,11,8,11
True 0,5,18,20
True 1,15,16,16


In [34]:
support_df

,Class,Support
0,-1,30
1,0,43
2,1,47


In [35]:
def evaluate_model_vs_rw(model_preds_csv, block_len=10, B=4000, seed=2025):
    model_df = pd.read_csv(model_preds_csv)
    # Align on Week & split_id to the RW OOS set
    merged = pd.merge(rw_preds[['Week','split_id','y_true','y_pred']].rename(columns={'y_pred':'y_pred_rw'}),
                      model_df[['Week','split_id','y_pred']].rename(columns={'y_pred':'y_pred_model'}),
                      on=['Week','split_id'], how='inner')
    y_true = merged['y_true'].astype(int).to_numpy()
    y_rw = merged['y_pred_rw'].astype(int).to_numpy()
    y_model = merged['y_pred_model'].astype(int).to_numpy()
    
    # Pooled metrics for the model + CIs
    pt, lo, hi = mbb_ci(y_true, y_model, block_len=block_len, B=B, seed=seed)
    pooled_ci = pd.DataFrame({
        "Metric": ["Accuracy", "Weighted F1", "Macro F1", "MCC"],
        "Point": pt,
        "CI Lower (95%)": lo,
        "CI Upper (95%)": hi,
        "n_weeks": [len(y_true)]*4
    })
    
    # McNemar: model vs RW on hits
    mc = mcnemar_test(y_true, y_model, y_rw)
    
    # DM test: 0-1 loss series
    loss_model = (y_true != y_model).astype(int)
    loss_rw = (y_true != y_rw).astype(int)
    dm = dm_test(loss_model, loss_rw, h=1, lag=0)
    
    # Package
    bundle = {
        "pooled_ci": pooled_ci,
        "mcnemar": mc,
        "dm": dm,
        "aligned_rows": len(merged)
    }
    return bundle

Headline numbers (pooled OOS, 95% CI):

Accuracy: 0.375 (CI: 0.30–0.45)

Weighted-F1: 0.375 (CI: 0.302–0.448)

Macro-F1: 0.375 (CI: 0.295–0.440)

MCC: 0.048 (CI: −0.082–0.151)

Interpretation for the thesis
The random-walk accuracy sits around 37–38% across 120 weeks on the 3-class task, with a fairly wide CI.

MCC’s CI straddles zero, which is exactly what you expect from a no-skill baseline: no reliable association with the true directions.

This is useful: it gives you a calibrated yardstick. Any model you present should beat these intervals, and you’ll be able to show that the lift is both statistically and economically meaningful.

2) Per-split variability (with CIs)
Per split, test windows are ~23 weeks (last block ~5 weeks), so CIs are wider and some splits dip negative on MCC. That swinginess is expected given small n and the 3-class set-up; it’s the main reason we’re doing rolling-origin rather than a single hold-out.